In [91]:
#Import our necessary python libraries
import pandas as pd
import requests
import json
import os
from collections import defaultdict
from IPython.display import HTML
import plotly.express as px
BLS_API_KEY  = 'd8644c9411c54f5d8333afc4185af8e1'
BLS_ENDPOINT = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
%matplotlib inline

In [136]:
#Grab our data from covidtracking.com through their API.
df_states_current = pd.read_json("https://covidtracking.com/api/states")
df_in_the_press   = pd.read_json("https://covidtracking.com/api/press")
df_us_current     = pd.read_json("https://covidtracking.com/api/us")
df_us_daily       = pd.read_json("https://covidtracking.com/api/us/daily")
df_counties       = pd.read_json("https://covidtracking.com/api/counties")


In [153]:
def get_bls_data(series, start, end):
    headers = {'Content-Type': 'application/json'}
    data = json.dumps({"seriesid": series,'registrationKey': 'd8644c9411c54f5d8333afc4185af8e1', "startyear":"%d" % (start), "endyear":"%d" % (end)})
    p = requests.post(BLS_ENDPOINT, data=data, headers=headers)
    json_data = json.loads(p.text)
    try:
        df = pd.DataFrame()
        for series in json_data['Results']['series']:
            df_initial = pd.DataFrame(series)
            series_col = df_initial['seriesID'][0]
            for i in range(0, len(df_initial) - 1):
                df_row = pd.DataFrame(df_initial['data'][i])
                df_row['seriesID'] = series_col
                if 'code' not in str(df_row['footnotes']): 
                    df_row['footnotes'] = ''
                else:
                    df_row['footnotes'] = str(df_row['footnotes']).split("'code': '",1)[1][:1]
                df = df.append(df_row, ignore_index=True)
        return df
    except:
        json_data['status'] == 'REQUEST_NOT_PROCESSED'
        print('BLS API has given the following Response:', json_data['status'])
        print('Reason:', json_data['message'])


start = 2019
end = 2020

In [158]:

# series = ['LNS14000000', 'LNU05026645', 'LNS12032194', 'LNS14027689']
# unemp = ['LASST340000000000003','LASST340000000000004','LASST340000000000005','LASST340000000000006','LASST340000000000007','LASST340000000000008']
unemp = ['LASST340000000000004']

df_unemployment = get_bls_data(series=unemp, start=start, end=end)

In [159]:
df_unemployment

,year,period,periodName,latest,value,footnotes,seriesID
0,2020,M03,March,true,174263,P,LASST340000000000004
1,2020,M02,February,NaN,173873,,LASST340000000000004
2,2020,M01,January,NaN,172261,,LASST340000000000004
3,2019,M12,December,NaN,167704,R,LASST340000000000004
4,2019,M11,November,NaN,168168,R,LASST340000000000004
5,2019,M10,October,NaN,167107,R,LASST340000000000004
6,2019,M09,September,NaN,161982,R,LASST340000000000004
7,2019,M08,August,NaN,155492,R,LASST340000000000004
8,2019,M07,July,NaN,149536,R,LASST340000000000004
9,2019,M06,June,NaN,146273,R,LASST340000000000004


In [141]:
df_us_daily[["date","total"]]


,date,total
0,20200501,6553449
1,20200430,6249467
2,20200429,6031002
3,20200428,5799934
4,20200427,5597572
...,...,...
96,20200126,1
97,20200125,1
98,20200124,1
99,20200123,1


In [ ]:
#Choropleth Map of positive COVID-19 Cases in the United States
statesDF = df_states_current[["state","positive"]]
fig = px.choropleth(locations=statesDF.state, locationmode="USA-states", color=statesDF.positive, scope="usa",
                   labels={'color':'Positive COVID-19 Cases'})
fig.update_layout(title='COVID-19 Virus Positive Cases By State (Reported)')
fig.show()

In [ ]:
df_counties

In [23]:
import requests
import json
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2014"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
    

In [24]:
json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 133,
 'message': [],
 'Results': {'series': [{'seriesID': 'CUUR0000SA0',
    'data': [{'year': '2014',
      'period': 'M12',
      'periodName': 'December',
      'value': '234.812',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M11',
      'periodName': 'November',
      'value': '236.151',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M10',
      'periodName': 'October',
      'value': '237.433',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M09',
      'periodName': 'September',
      'value': '238.031',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M08',
      'periodName': 'August',
      'value': '237.852',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M07',
      'periodName': 'July',
      'value': '238.250',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M06',
      'periodName': 'June',
      'value': '238.343',
   